# Kamiwaza Doc Ingestion/Embedding/Vector searching

## Overview

## Ingesting wikipedia

source: https://huggingface.co/datasets/rungalileo/ragbench

This can be done similar to this:

```
service = IngestionService('sec_data')
service.ingest_and_embed(
    's3://kamiwaza-demo/pdfs/parsed',
    description='SEC 10-K Filings',
    # we pass none because we are assuming the catalog has these already,
    # so this is just tagging the secrets to these catalog entries
    secret_access_key = {'cloudflare_secret_access_key': None},
    access_key_id = {'cloudflare_access_key_id': None},
    endpoint_url = {'cloudflare_endpoint': None},
    region = {'cloudflare_region': None}
)
```

However, for the sake of showing more capabilities step-by-step, we are doing this a bit more step by step


In [1]:
# just various imports; some of these are used later, but we import them all now so our
# logger adjustments can affect them
import logging
logging.basicConfig(level=logging.WARNING)
from kamiwaza.services.catalog.services import DataService
from kamiwaza.middleware.embedding.sentencetransformers import SentenceTransformerEmbedding
from kamiwaza.middleware.vectordb.milvus import MilvusImplementation
from typing import List, Dict, Any, Sequence
import numpy as np

import glob
import sys
import ray

# be less noisy
logging.getLogger("requests").setLevel(logging.WARNING)
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.WARNING)



# This will print the path to the Python interpreter being used,
# which usually includes the name of the virtual environment
print("Current Virtual Environment:", sys.executable)
default_model_name=SentenceTransformerEmbedding().model_name
print(f"Default embedding model: {default_model_name}")

DOCS_PATH='/mnt/data/ragbench'



Current Virtual Environment: /home/kamiwaza/kamiwaza/notebook-venv/bin/python
Default embedding model: BAAI/llm-embedder


### Suppress verbose logging

In [2]:
# This cell is basically 100% just to suppress noisy debug messages
# This should work across all nodes in a cluster when using Ray

from typing import List
from ray.util.scheduling_strategies import NodeAffinitySchedulingStrategy

# For Jupyter, dispense with the super-verbose debugging
def set_logging_level(level=logging.WARNING):
    logging.getLogger().setLevel(level)

#### Ray Init - None of this is functionally required; Ray will turn on automatically if
#### needed when called; however, we explicitly initialize it here to lower logging to
#### the verbose logging, as notebook settings will not carry into ray since those
#### environments are totally encapsulated (even if local)
if not ray.is_initialized():
    # Example of manually setting required PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION
    # not needed if you started ray with Kamiwaza start-ray.sh but if you have
    # an existing or self-managed ray cluster, this may be needed
    runtime_env = {
        "env_vars": {
            "PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION": "python"
        }
    }
    ray.init(configure_logging=True, logging_level=logging.ERROR, runtime_env=runtime_env)

@ray.remote
def init_worker():
    set_logging_level()

@ray.remote
def node_affinity_func():
    return ray.get_runtime_context().get_node_id()

# Get list of nodes
nodes: List[str] = ray.nodes()

# Run init_worker on each node
for node in nodes:
    node_id = node.get('NodeID')
    init_worker.options(
        scheduling_strategy=NodeAffinitySchedulingStrategy(
            node_id=node_id,
            soft=False,
        )
    ).remote()

# Set the level for all loggers to WARNING
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.WARNING)

### Step 1: Add the location of your data to the catalog. (In this case, it's just a path)

In [3]:
COLLECTION_NAME='ragbench' # name for vectordb collection

In [4]:

# Get a dataservice handle with the datahub plugin (Kamiwaza Default)
data_service = DataService('data_datahub') # not yet a default, but may become one

# Super simple catalog ingestion
catalog_items = data_service.create_dataset(
    dataset_name=DOCS_PATH,
    platform='file', # s3 is the other major platform, for any s3-compatible object; gcs and azure WIP
    description='Wikipedia Dense Passage Retrieval Tests',
    additional_properties = {'collection': COLLECTION_NAME} # Kamiwaza ingesters can set this automatically, but we will set it here
)

print("Ingested")

Ingested


### Step 2: Retrieve Datasets

In [5]:
# moved these up top to silence debug loggings
#from kamiwaza.services.retrieval.runners import FileRunner
#from kamiwaza.middleware.embedding import SentenceTransformerEmbedding
#from kamiwaza.middleware.vectordb import MilvusImplementation


# we will retrieve the urn of the dataset
catalog_urn = catalog_items


In [6]:



fields = [
    #('model_name', str, default_model_name),
    #('source', str)
    #('catalog_urn', str) # using Kamiwaza ingest services this is done for you
    #('srcid', str), # id we add since this is in the parquet data

]

vectordb = MilvusImplementation()

# kamiwaza add scheme will call milvus.client.add_schema but normalize with an uuid index named id and an embedding of supplied dims
# if we were unsure of the dims, we can instantiate SentenceTransformers(model=MODEL_NAME) ahead of time and check the .dims property
# eg
# embedder = SentenceTransformers() #default is BAAI/llm-embedder at time of writing, likely to change
vectordb.drop_schema(COLLECTION_NAME)

# we could interrogate dims from our model if we didn't want to hardcode it
vectordb.add_schema(COLLECTION_NAME, dims=768, fieldlist=fields)


{'result': True,
 'collection': <Collection>:
 -------------
 <name>: ragbench
 <description>: ragbench Collection
 <schema>: {'auto_id': True, 'description': 'ragbench Collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}, {'name': 'model_name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}, {'name': 'source', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}, {'name': 'catalog_urn', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}, {'name': 'offset', 'description': '', 'type': <DataType.INT64: 5>}], 'enable_dynamic_field': False}}

In [7]:
glob.glob(DOCS_PATH+"/**/*.parquet")

['/mnt/data/ragbench/cuad/train-00000-of-00001.parquet',
 '/mnt/data/ragbench/cuad/validation-00000-of-00001.parquet',
 '/mnt/data/ragbench/cuad/test-00000-of-00001.parquet',
 '/mnt/data/ragbench/hotpotqa/train-00000-of-00001.parquet',
 '/mnt/data/ragbench/hotpotqa/validation-00000-of-00001.parquet',
 '/mnt/data/ragbench/hotpotqa/test-00000-of-00001.parquet',
 '/mnt/data/ragbench/delucionqa/train-00000-of-00001.parquet',
 '/mnt/data/ragbench/delucionqa/validation-00000-of-00001.parquet',
 '/mnt/data/ragbench/delucionqa/test-00000-of-00001.parquet',
 '/mnt/data/ragbench/covidqa/train-00000-of-00001.parquet',
 '/mnt/data/ragbench/covidqa/validation-00000-of-00001.parquet',
 '/mnt/data/ragbench/covidqa/test-00000-of-00001.parquet',
 '/mnt/data/ragbench/msmarco/train-00000-of-00001.parquet',
 '/mnt/data/ragbench/msmarco/validation-00000-of-00001.parquet',
 '/mnt/data/ragbench/msmarco/test-00000-of-00001.parquet',
 '/mnt/data/ragbench/tatqa/train-00000-of-00001.parquet',
 '/mnt/data/ragbenc

In [8]:
# First load the HuggingFace dataset normally
ds = ray.data.read_parquet(glob.glob(DOCS_PATH+"/**/*.parquet"))


Metadata Fetch Progress 0:   0%|          | 0/6 [00:00<?, ?it/s]

Parquet Files Sample 0:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
ds.schema()

Column                              Type
------                              ----
id                                  string
question                            string
documents                           list<element: string>
response                            string
generation_model_name               string
annotating_model_name               string
dataset_name                        string
documents_sentences                 list<element: list<element: list<element: string>>>
response_sentences                  list<element: list<element: string>>
sentence_support_information        list<element: struct<explanation: string, fully_supported: bool, response_sentence_key: string, supporting_sentence_keys: list<element: string>>>
unsupported_response_sentence_keys  list<element: string>
adherence_score                     bool
overall_supported_explanation       string
relevance_explanation               string
all_relevant_sentence_keys          list<element: string>
all_utilized_sen

In [10]:
catalog_urn

'urn:li:dataset:(urn:li:dataPlatform:file,/mnt/data/ragbench,PROD)'

In [11]:
ds.count()

95381

In [12]:
def remap_columns(batch: Dict[str, np.ndarray], catalog_urn: str) -> Dict[str, np.ndarray]:
    def process_id(id_str):
        if not id_str:
            return "na"
        id_str = str(id_str)
        return id_str[:1000] if len(id_str) > 1000 else id_str

    batch_size = len(batch['id'])
    
    # Initialize lists of correct length
    result = {
        'text': [''] * batch_size,
        'srcid': [''] * batch_size,
        'source': [''] * batch_size,
        'catalog_urn': [catalog_urn] * batch_size,
        'offset': [0] * batch_size,
    }
    
    # Fill in each position
    for i in range(batch_size):
        result['text'][i] = '\n\n'.join(batch['documents'][i])
        result['srcid'][i] = process_id(batch['id'][i])
        result['source'][i] = process_id(batch['id'][i])

    return result

In [13]:
class SentenceTransformerActor:
    def __init__(self, model: str = 'BAAI/llm-embedder', device: str = None, max_operations=10000):
        self.embedder = SentenceTransformerEmbedding()
        self.opcount = 0
        self.MAX_OPCOUNT = max_operations

    def __call__(self, batch, **kwargs):
        self.opcount += 1
        print("opcount: ", self.opcount)
        result = self.embedder.__call__(batch, **kwargs)
        if self.opcount >= self.MAX_OPCOUNT:
            self.self_terminate()
        return result

    def self_terminate(self):
        ray.actor.exit_actor()

In [ ]:
# an "expanded" version of the example - this is code
import pyarrow as pa
import ray.data

fields = pa.schema([
    ("id", pa.string()),
    ("documents", pa.list_(pa.string()))
])
# First load the HuggingFace dataset normally
ds = ray.data.read_parquet(glob.glob(DOCS_PATH+"/**/train*.parquet"), columns=["id", "documents"])

print(ds.schema())
import psutil

# Lots of ways to slice our resources but this should be reasonable and we will let both embeddings and vector use ~45% of cores
num_cpus = int(psutil.cpu_count() * 0.45)

# Enable GPU - .01 for gpu free-for-all
num_gpus = 0.20 if ray.available_resources().get('GPU', None) else None

# Remap since SentenceTransformerEmbedding expects text and preamble columns for semantics, not text and title
mapped_dataset = ds.map_batches(remap_columns, fn_kwargs={'catalog_urn': catalog_urn}, concurrency=4, batch_size=None)


# generate embeddings!
embedded_dataset = mapped_dataset.map_batches(
    SentenceTransformerActor,
    concurrency=(1,16),
    num_gpus = num_gpus,
    batch_size=1024, 
    zero_copy_batch=True, # when not mutating input, this is faster
)

# insert embeddings into vectordb
milvus_dataset = embedded_dataset.map_batches(
    MilvusImplementation,
    concurrency=(1,4),
    num_gpus = num_gpus,
    batch_size=1024, # we will make this slightly larger to catch splits even though most entries are quite small
    zero_copy_batch=True,
    fn_kwargs={'collection_name': COLLECTION_NAME}
)

# Step 5: Take all results (this materializes the pipeline, as would .materialize() - without that things would not begin to evaluate)
results = milvus_dataset.take_all()
len(results)


Parquet Files Sample 0:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
results

## Now we have searchable vectors

In the examples below we do various vector searches. It's important to note that:

1. Dense embeddings aren't always the best option for retrieval; sparse embeddings (like SPLADE) or bag-or-words representations with reverse-index search like BM25 are also very valuable
2. An upcoming Kamiwaza release will automate hybrid vectors or hybrid vector+reverse index searching
3. Kamiwaza's retrieval engine will have an auto-reranked in the near future; that will allow you to retrieve a larger context of docs (say, 20 or 50 instead of 5 or 10) and allow Kamiwaza to then rerank the chunks specifically against the user query; this also works very well with BM25 and Sparse vectors
4. We've done ~nothing to optimize here. We could have:

* Stripped the redundant copyright notice
* Found the doc title and put it in the preamble
* Actually chunked on headers/subheaders (and added subheaders to the preamble)
* etc

Exercise left to the reader for now!

In [ ]:
# Simple search
# Note: this flow of 'embed the query and get back results' will be reduced to a single method in the future in kamiwaza middleware

st = SentenceTransformerEmbedding()
query = "Represent this query for retrieving relevant documents: what are the params for AutoModel.from_pretrained()?"
# For the record, in our tests, the 3rd result here (main_classes/model.md offset 1894) is what we
# imagined getting back; you'll also possibly see 2 results with the same path and offset 0;
# which means that there were two different embeddings from that file that were highly relevant, but
# the tokenizer had issues with some passages or kamiwaza had a problem aligning them (this is
# more likely with dense passages of code, as it is more likely to have large strings of '##suffix' tokens
# and with SentenceTransformers, we must align tokens and text ourselves

# usecase and mode allow Kamiwaza to map to model instructions if available
# some models, especially for query embeddings, expect an instruction like
# "Represent this passage for searching relevent texts: " etc
query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector(COLLECTION_NAME, query_embedding, output_fields=["*"])

for i in range(0,min(10,len(search_results[0])),1):
    print(f"{search_results[0].distances[i]} {search_results[0][i].entity.get('source')} offset {search_results[0][i].entity.get('offset')}")



In [ ]:
# Search of a specific fact - we use this in the AgentZero retrieval demo!

st = SentenceTransformerEmbedding()
query = "who wrote the paper that presented the MBart model??"

# as an aside, without our header-stripping in the ProcessDoc code, first chunk of the umt5.md doc
# is not semantically strong enough for the embedding model to correctly associate it with the query,
# despite the fact that the first real readable text under the title is
# "The UMT5 model was proposed in UniMax: Fairer and More Effective Language Sampling for Large-Scale Multilingual Pretraining by
# Hyung Won Chung, Xavier Garcia, Adam Roberts, Yi Tay, Orhan Firat, Sharan Narang, Noah Constant."

# Reams could be written on this in the micro and how it points toward hybrid searching, entity extraction, etc.

query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector(COLLECTION_NAME, query_embedding, output_fields=["*"])

for i in range(0,min(10,len(search_results[0])),1):
    print(f"{search_results[0].distances[i]} {search_results[0][i].entity.get('source')} offset {search_results[0][i].entity.get('offset')}")



In [ ]:
# verbatim search search
# here we query with an exact passage from the umt5.md page; this of course is what we'd expect to be the first result
# but shockingly similar how other embeddings are - although llm-embedder has a similarity distribution
# of [0.6,1] because of the contrastive learning approach;

st = SentenceTransformerEmbedding()

# Verbatim text selecting from from /model_doc/umt5.md
query = "As part of our contribution, we release: (i) an improved and refreshed mC4 multilingual corpus consisting of 29 trillion characters across 107 languages, and (ii) a suite of pretrained umT5 model checkpoints trained with UniMax sampling."

# usecase and mode allow Kmiwaza to map to model instructions if available
# some models, especially for query embeddings, expect an instruction like
# "Represent this passage for searching relevent texts: " etc
query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector(COLLECTION_NAME, query_embedding, output_fields=["*"])

for i in range(0,min(10,len(search_results[0])),1):
    print(f"{search_results[0].distances[i]} {search_results[0][i].entity.get('source')} offset {search_results[0][i].entity.get('offset')}")


In [ ]:
# unrelated search query; should return a lower relevance; probably < .8 with Milvus

# here we search something utterly unrelated; however, dense embeddings often have
# some random overlap between vectors; it can be from words, phrases, sentence
# structure

st = SentenceTransformerEmbedding()
query = "What movie paired Sandra Bullock and Wesley Snipes early in her career?"

# usecase and mode allow Kmiwaza to map to model instructions if available
# some models, especially for query embeddings, expect an instruction like
# "Represent this passage for searching relevent texts: " etc
query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector(COLLECTION_NAME, query_embedding, output_fields=["*"])

for i in range(0,min(10,len(search_results[0])),1):
    print(f"{search_results[0].distances[i]} {search_results[0][i].entity.get('source')} offset {search_results[0][i].entity.get('offset')}")
